In [5]:
import itertools
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.cluster import KMeans 
import warnings
from mpl_toolkits.mplot3d.art3d import Poly3DCollection


def create_grid(length, width, spacing=1):
    """
    Create a grid on the ceiling or floor based on room dimensions.

    :param length: Length of the room
    :param width: Width of the room
    :param spacing: Spacing between grid points
    :return: List of grid points
    """
    x = np.arange(0, length + spacing, spacing)
    y = np.arange(0, width + spacing, spacing)
    grid = np.array(np.meshgrid(x, y)).T.reshape(-1, 2)
    return grid   

def create_object(lower_corner, length, width, height=0.1):  # Default height is 0
    """
    Create a representation of a rectangular prism object in the room.

    :param lower_corner: The (x, y, z) coordinates of the lower corner of the object
    :param length: The length of the object
    :param width: The width of the object
    :param height: The height of the object (default is 0)
    :return: A dictionary representing the object
    """
    return {
        'lower_corner': np.array(lower_corner),
        'length': length,
        'width': width,
        'height': height
    }

def calculate_shadow_impact(light_pos, floor_grid, objects):
    """
    Calculate the impact of shadows cast by objects for a given light position.

    :param light_pos: The position of the light (3D)
    :param floor_grid: Grid points on the floor
    :param objects: List of objects in the room
    :return: Percentage of the floor grid not in shadow
    """
    shadowed_points = sum(is_in_shadow(floor_pos, light_pos, objects) for floor_pos in floor_grid)
    total_points = len(floor_grid)
    return 1 - shadowed_points / total_points  # Percentage of non-shadowed floor area 



def point_in_plane(point, plane):
    """
    Determine if a given point is within a plane defined by three points.

    This function calculates whether a point lies within the bounds of a plane.
    The plane is defined by three points (forming two vectors on the plane), and the check involves:
    1. Determining if the point is on the plane by checking if the vector from the plane to the point 
       is orthogonal to the normal vector of the plane.
    2. Checking if the point lies within the rectangular bounds defined by the plane's vertices.

    Args:
        point (numpy.ndarray): A 3D point represented as [x, y, z].
        plane (list[numpy.ndarray]): A list of three 3D points (each as [x, y, z]) that define the plane.

    Returns:
        bool: True if the point is within the plane, False otherwise.
    """

    normal_vector = np.cross((plane[1] - plane[0]), (plane[2] - plane[0]))
    vector_to_point = point - plane[0]
    dot_product = np.dot(vector_to_point, normal_vector)
    epsilon = 1e-6  # Tolerance for checking if point is in plane

    if abs(dot_product) > epsilon:  # Adjusted check
        return False  # Point is not in the plane

    # Check if point is within the bounds of the plane
    plane_x = [p[0] for p in plane]
    plane_y = [p[1] for p in plane]
    plane_z = [p[2] for p in plane]
    return (min(plane_x) <= point[0] <= max(plane_x) and
            min(plane_y) <= point[1] <= max(plane_y) and
            min(plane_z) <= point[2] <= max(plane_z))

    
def create_planes(obj):
    """
    Defines the planes of an object as an array of four vertices for each plane.

    Args:
        obj: A dictionary representing the object with keys 'lower_corner', 'length', 'width', and 'height'.

    Returns:
        An array of planes, where each plane is represented by four vertices (x, y, z).
    """

    lower_corner = obj['lower_corner']
    length = obj['length']
    width = obj['width']
    height = obj['height']

    # Define vertices for each plane (bottom, top, front, back, right, left)
    vertices = np.array([
        [lower_corner[0], lower_corner[1], lower_corner[2]],
        [lower_corner[0] + length, lower_corner[1], lower_corner[2]],
        [lower_corner[0] + length, lower_corner[1] + width, lower_corner[2]],
        [lower_corner[0], lower_corner[1] + width, lower_corner[2]],
        [lower_corner[0], lower_corner[1], lower_corner[2] + height],
        [lower_corner[0] + length, lower_corner[1], lower_corner[2] + height],
        [lower_corner[0] + length, lower_corner[1] + width, lower_corner[2] + height],
        [lower_corner[0], lower_corner[1] + width, lower_corner[2] + height]
    ])

    # Define planes as sets of four vertices for each face
    planes = [
        vertices[[0, 1, 2, 3]],
        vertices[[4, 5, 6, 7]],
        vertices[[0, 1, 5, 4]],
        vertices[[2, 3, 7, 6]],
        vertices[[1, 2, 6, 5]],
        vertices[[0, 3, 7, 4]]
    ]

    return planes

def line_intersects_plane(p1, p2, plane):
    
    """
    Determine if a line segment intersects with a plane and find the intersection point.

    This function checks if the line segment defined by points p1 and p2 intersects with a plane defined by three points. 
    It calculates the intersection point using vector and plane equations.

    Args:
        p1 (numpy.ndarray): The starting point of the line segment [x, y, z].
        p2 (numpy.ndarray): The ending point of the line segment [x, y, z].
        plane (list[numpy.ndarray]): Three 3D points defining the plane.

    Returns:
        numpy.ndarray or None: The intersection point if it exists; otherwise, None.
    """
        
    p1 = np.array(p1)
    p2 = np.array(p2)
    direction_vector = p2 - p1
    normal_vector = np.cross((plane[1] - plane[0]), (plane[2] - plane[0]))

    dot_product = np.dot(direction_vector, normal_vector)
    epsilon = 1e-6  # A small tolerance value

    if abs(dot_product) < epsilon:  # Adjusted check for parallelism
        return None  # Line is parallel or almost parallel to the plane

    t = -np.dot(normal_vector, p1 - plane[0]) / dot_product
    if 0 <= t <= 1:
        return p1 + t * direction_vector
    else:
        return None


def is_in_shadow(floor_pos, light_pos, objects):
    """
    Determine if a point on the floor is in the shadow of any objects in the room.

    This function checks each object in the room to determine if a line from the light source to the floor point intersects with it, 
    indicating the point is in shadow.

    Args:
        floor_pos (tuple): The x, y coordinates of the point on the floor.
        light_pos (tuple): The x, y, z coordinates of the light source.
        objects (list[dict]): A list of objects (dicts) in the room, each defined by its lower corner, length, width, and height.

    Returns:
        bool: True if the point is in shadow, False otherwise.
    """
    floor_x, floor_y = floor_pos
    light_x, light_y, light_z = light_pos 
    
    for obj in objects:
        obj_planes = create_planes(obj)
        for plane in obj_planes:
            intersection = line_intersects_plane([light_x, light_y, light_z], [floor_x, floor_y, 0], plane)
            if intersection is not None and point_in_plane(intersection, plane):
                return True
    return False




def apply_kmeans_clustering(ceiling_grid, num_clusters): 
    """
    Apply KMeans clustering algorithm to group ceiling grid points.

    This function uses the KMeans clustering algorithm to group grid points on the ceiling into clusters. 
    This is typically used to determine initial positions for lights in a room.

    Args:
        ceiling_grid (numpy.ndarray): Grid points on the ceiling.
        num_clusters (int): The number of clusters (or lights) to form.

    Returns:
        numpy.ndarray: The centers of the formed clusters.
    """
    kmeans = KMeans(n_clusters=num_clusters, random_state=0).fit(ceiling_grid)
    cluster_centers = kmeans.cluster_centers_
    return cluster_centers


def calculate_illuminance(light_positions, light_intensities, floor_grid, ceiling_height, objects):
    """
    Calculate illuminance at each floor grid point considering shadows from objects.

    :param light_positions: Positions of the lights (2D)
    :param light_intensities: Intensities of the lights
    :param floor_grid: Grid points on the floor
    :param ceiling_height: Height of the ceiling
    :param objects: List of objects in the room
    :return: Array of illuminance values at each floor grid point
    """
    illuminance_floor = np.zeros(len(floor_grid))
    
    for light_pos, light_intensity in zip(light_positions, light_intensities):
        # Add the ceiling height as the z-coordinate for the light position
        light_pos_3D = np.array([light_pos[0], light_pos[1], ceiling_height])

        for i, floor_pos in enumerate(floor_grid):
            if not is_in_shadow(floor_pos, light_pos_3D, objects):
                distance = np.sqrt((light_pos_3D[0] - floor_pos[0])**2 + (light_pos_3D[1] - floor_pos[1])**2 + ceiling_height**2)
                illuminance_floor[i] += light_intensity / (distance**2)

    return illuminance_floor


def optimize_within_cluster(cluster_center, ceiling_grid, floor_grid, ceiling_height, light_intensity, objects): 
    """
    Optimize the position of a light within its cluster to maximize illuminance while considering shadows.

    This function searches within a specified radius around a cluster center to find the best position for a light. 
    It evaluates each potential position based on average illuminance, standard deviation of illuminance, 
    and shadow impact, choosing the position that optimizes these factors.

    Args:
        cluster_center (numpy.ndarray): The center of the cluster (3D point).
        ceiling_grid (numpy.ndarray): Grid points on the ceiling.
        floor_grid (numpy.ndarray): Grid points on the floor.
        ceiling_height (float): Height of the ceiling.
        light_intensity (float): Intensity of the light.
        objects (list[dict]): List of objects in the room affecting shadows.

    Returns:
        numpy.ndarray: The best position for the light within the cluster.
    """
    
    best_position = None
    best_illuminance = float('-inf')
    best_std_dev = float('inf')
    best_shadow_impact = float('-inf')  # Initialize the best shadow impact

    search_radius = max(1, spacing)

    for point in ceiling_grid:
        cluster_center_2d = cluster_center[:2]
        if np.linalg.norm(point[:2] - cluster_center_2d) <= search_radius:
            light_pos_3D = np.array([point[0], point[1], ceiling_height])
            illuminance = calculate_illuminance([point], [light_intensity], floor_grid, ceiling_height, objects)
            avg_illuminance = np.mean(illuminance)
            std_dev = np.std(illuminance)
            shadow_impact = calculate_shadow_impact(light_pos_3D, floor_grid, objects)

            # Check for improvements considering shadow impact
            if (avg_illuminance > best_illuminance or 
                (avg_illuminance == best_illuminance and std_dev < best_std_dev) or
                (avg_illuminance == best_illuminance and std_dev == best_std_dev and shadow_impact > best_shadow_impact)):
                best_illuminance = avg_illuminance
                best_std_dev = std_dev
                best_shadow_impact = shadow_impact
                best_position = point

    return best_position




def find_optimal_lighting_setup(ceiling_grid, floor_grid, num_lights, ceiling_height, light_intensities, objects):
    """
    Find the optimal setup of multiple lights in the room considering shadows, illuminance, and permutations of light intensities.

    This function first applies KMeans clustering to determine initial positions for the lights. 
    It then iterates through permutations of light intensities, optimizing the position of each light within its cluster 
    to maximize overall illuminance and minimize shadow impact. The best combination of positions and intensities 
    that yields the highest average illuminance is selected.

    Args:
        ceiling_grid (numpy.ndarray): Grid points on the ceiling.
        floor_grid (numpy.ndarray): Grid points on the floor.
        num_lights (int): Number of lights to be placed.
        ceiling_height (float): Height of the ceiling.
        light_intensities (list[float]): List of intensities for each light.
        objects (list[dict]): List of objects in the room affecting shadows.

    Returns:
        list[numpy.ndarray]: Optimal positions for each light.
        list[float]: Corresponding intensities for each light.
        float: Highest average illuminance achieved.
        float: Standard deviation of illuminance for the optimal setup.
    """
    best_overall_positions = []
    best_overall_intensities = []
    best_overall_avg_illuminance = float('-inf')
    best_overall_std_dev = float('inf')

    # Apply clustering to find initial light positions
    cluster_centers = apply_kmeans_clustering(ceiling_grid, num_lights)
    cluster_centers_3D = [np.append(center, ceiling_height) for center in cluster_centers]

    # Iterate through all permutations of light intensities
    for permutation in itertools.permutations(light_intensities):
        current_positions = []
        for i, cluster_center in enumerate(cluster_centers_3D):
            # Optimize position for each light within its cluster
            best_position = optimize_within_cluster(cluster_center, ceiling_grid, floor_grid, ceiling_height, permutation[i], objects)
            if best_position is not None:
                current_positions.append(best_position)

        # Calculate average illuminance and standard deviation for these positions
        if current_positions:
            illuminance = calculate_illuminance(current_positions, permutation, floor_grid, ceiling_height, objects)
            avg_illuminance = np.mean(illuminance)
            std_dev = np.std(illuminance)

            # Update if this permutation yields a better result
            if avg_illuminance > best_overall_avg_illuminance or (avg_illuminance == best_overall_avg_illuminance and std_dev < best_overall_std_dev):
                best_overall_positions = current_positions
                best_overall_intensities = permutation
                best_overall_avg_illuminance = avg_illuminance
                best_overall_std_dev = std_dev

    return best_overall_positions, list(best_overall_intensities), best_overall_avg_illuminance, best_overall_std_dev

SyntaxError: invalid syntax (4102015590.py, line 234)

In [ ]:
def plot_light_positions_in_room(room_length, room_width, room_height, light_positions, light_intensities):
    """
    Plot the best light positions in a 3D representation of the room.

    :param room_length: Length of the room
    :param room_width: Width of the room
    :param room_height: Height of the room
    :param light_positions: Positions of the best lights
    """
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    # Plot the room boundaries
    for x in [0, room_length]:
        for y in [0, room_width]:
            ax.plot([x, x], [y, y], [0, room_height], color="b")

    # Plot the ceiling and the floor
    ceiling_floor_x = [0, room_length, room_length, 0, 0]
    ceiling_floor_y = [0, 0, room_width, room_width, 0]
    ax.plot(ceiling_floor_x, ceiling_floor_y, 0, color="g")
    ax.plot(ceiling_floor_x, ceiling_floor_y, room_height, color="r")

     # Normalize the light intensities for color mapping
    norm = plt.Normalize(min(light_intensities), max(light_intensities))
    cmap = plt.cm.viridis

    for pos, intensity in zip(light_positions, light_intensities):
        color = cmap(norm(intensity))
        ax.scatter(pos[0], pos[1], room_height, color=color, s=100)
        ax.text(pos[0], pos[1], room_height, f'{intensity}', color='black')

    ax.set_xlabel('X axis')
    ax.set_ylabel('Y axis')
    ax.set_zlabel('Z axis')
    plt.title('3D Visualization of Light Positions in Room')
    plt.show()

    
def plot_light_positions_and_heatmap(room_length, room_width, room_height, light_positions, light_intensities, ceiling_grid, floor_grid, spacing, objects):
    """
    Plot the best light positions in a 3D representation of the room and a heatmap of illuminance levels on the floor.

    :param room_length: Length of the room
    :param room_width: Width of the room
    :param room_height: Height of the room
    :param light_positions: Positions of the best lights
    :param light_intensities: Array of illuminance levels for all lights
    :param ceiling_grid: Grid points on the ceiling
    :param floor_grid: Grid points on the floor
    :param spacing: Spacing between grid points
    :param objects: List of objects in the room
    """ 

    # Calculate the illuminance for the floor grid
    illuminance_floor = calculate_illuminance(light_positions, light_intensities, floor_grid, room_height, objects)
    
    # Reshape the illuminance array to match the floor grid dimensions
    grid_length = int(np.ceil(room_length / spacing) + 1)
    grid_width = int(np.ceil(room_width / spacing) + 1)
    illuminance_reshape = illuminance_floor.reshape(grid_length, grid_width)

    # Create the 3D plot for room and light positions
    fig = plt.figure(figsize=(12, 6))
    ax1 = fig.add_subplot(121, projection='3d')

    # Plot the room boundaries
    for x in [0, room_length]:
        for y in [0, room_width]:
            ax1.plot([x, x], [y, y], [0, room_height], color="b")

    # Plot the ceiling and the floor
    ceiling_floor_x = [0, room_length, room_length, 0, 0]
    ceiling_floor_y = [0, 0, room_width, room_width, 0]
    ax1.plot(ceiling_floor_x, ceiling_floor_y, 0, color="g")
    ax1.plot(ceiling_floor_x, ceiling_floor_y, room_height, color="r")

    # Plot the light positions
    for pos in light_positions:
        ax1.scatter(pos[0], pos[1], room_height, color='y', s=100)  # Light positions on the ceiling

    ax1.set_xlabel('X axis')
    ax1.set_ylabel('Y axis')
    ax1.set_zlabel('Z axis')
    ax1.set_title('3D Visualization of Light Positions in Room')

    # Create the 2D heatmap for illuminance levels on the floor
    ax2 = fig.add_subplot(122)
    heatmap = ax2.imshow(illuminance_reshape, cmap='hot', interpolation='nearest', origin='lower')
    ax2.set_title('Heatmap of Illuminance Levels on the Floor')
    plt.colorbar(heatmap, ax=ax2, orientation='vertical')

    plt.tight_layout()
    plt.show()

    
def plot_room_with_grids_and_lights(room_length, room_width, room_height, ceiling_grid, floor_grid, light_positions, light_intensities, objects):
    """
    Plot the ceiling grid, floor grid, and best light positions in a 3D representation of the room along with objects.

    :param room_length: Length of the room
    :param room_width: Width of the room
    :param room_height: Height of the room
    :param ceiling_grid: Grid points on the ceiling
    :param floor_grid: Grid points on the floor
    :param light_positions: Positions of the best lights
    :param light_intensities: Intensities of the lights
    :param objects: List of objects in the room
    """
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')

    # Plot the room boundaries
    for x in [0, room_length]:
        for y in [0, room_width]:
            ax.plot([x, x], [y, y], [0, room_height], color="b")

    # Plot the ceiling and the floor grids
    for pos in ceiling_grid:
        ax.scatter(pos[0], pos[1], room_height, color='c', s=20)  # Ceiling grid points
    for pos in floor_grid:
        ax.scatter(pos[0], pos[1], 0, color='m', s=20)  # Floor grid points

    for obj in objects:
        lower_corner = obj['lower_corner']
        length = obj['length']
        width = obj['width']
        height = obj['height']

        # Vertices of the object
        vertices = np.array([
            [0, 0, 0],
            [length, 0, 0],
            [length, width, 0],
            [0, width, 0],
            [0, 0, height],
            [length, 0, height],
            [length, width, height],
            [0, width, height]
        ]) + lower_corner

        # Faces of the object
        faces = [
            [vertices[0], vertices[1], vertices[2], vertices[3]],  # Bottom
            [vertices[4], vertices[5], vertices[6], vertices[7]],  # Top
            [vertices[0], vertices[1], vertices[5], vertices[4]],  # Front
            [vertices[2], vertices[3], vertices[7], vertices[6]],  # Back
            [vertices[1], vertices[2], vertices[6], vertices[5]],  # Right
            [vertices[0], vertices[3], vertices[7], vertices[4]]   # Left
        ]

        obj_poly3d = Poly3DCollection(faces, alpha=0.7, facecolors='brown')
        ax.add_collection3d(obj_poly3d)

    # Plot the light positions
    for pos, intensity in zip(light_positions, light_intensities):
        ax.scatter(pos[0], pos[1], room_height, color='y', s=100, label=f'Intensity: {intensity}')

    ax.set_xlabel('X axis')
    ax.set_ylabel('Y axis')
    ax.set_zlabel('Z axis')
    plt.title('3D Visualization of Room with Ceiling and Floor Grids, Light Positions, and Objects')
    plt.legend()
    plt.show()


In [3]:
# Define some objects in the room
object1 = create_object(lower_corner=(1, 1, 0), length=1, width=1, height=2)  # Example object
object2 = create_object(lower_corner=(4, 2, 0), length=1, width=1, height=0.5)  # Another example object 

objects = []
 
light_intensities = [500,1000,10]     
    
# Continue with your existing setup
room_length = 5 # meters
room_width = 5   # meters
room_height = 2   # meters
num_lights = len(light_intensities) # Number of lights
spacing = 0.5
warnings.simplefilter(action='ignore', category=FutureWarning)


# Example usage with different light intensities for each light bulb
#light_intensities = [500,500,1000]   # Array of different light intensities 

# Create grids
ceiling_grid = create_grid(room_length, room_width, spacing) 
floor_grid = create_grid(room_length, room_width, spacing)   




optimal_positions, optimal_intensities, optimal_avg_illuminance, optimal_std_dev = find_optimal_lighting_setup(ceiling_grid, floor_grid, num_lights, room_height, light_intensities, objects)


# Print best light positions and their average illuminance and standard deviation
print("Best light positions:", optimal_positions)
print("Best light intensities:", optimal_intensities)
print("Average Illuminance:", optimal_avg_illuminance)
print("Standard Deviation of Illuminance:", optimal_std_dev)

plot_room_with_grids_and_lights(room_length, room_width, room_height, ceiling_grid, floor_grid, optimal_positions, optimal_intensities, objects)
plot_light_positions_in_room(room_length, room_width, room_height, optimal_positions, optimal_intensities)
plot_light_positions_and_heatmap(room_length, room_width, room_height, optimal_positions, optimal_intensities, ceiling_grid, floor_grid, spacing,objects)

NameError: name 'create_object' is not defined